## Azure AI Foundry Agent Service: Logic App tools

This notebook demonstrates the setup and use of Azure Logic Apps as a tool for Azure AI Foundry's Agent Service, using OpenAPI schema integration.

### Environment Setup

In [1]:
# Import required packages
import os
from typing import Dict, Any

from azure.identity import DefaultAzureCredential
from azure.mgmt.logic import LogicManagementClient
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    OpenApiTool, 
    OpenApiAnonymousAuthDetails
)

In [2]:
# Set environment variables
PROJECT_ENDPOINT = os.environ.get("AZURE_FOUNDRY_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.environ.get("AZURE_FOUNDRY_GPT_MODEL")
SUBSCRIPTION_ID = os.environ.get("AZURE_SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP_NAME", "App_LogicApp")

LOGIC_APP_NAME = "Laziz_DemoConsLA"  # Replace with your Logic App name
TRIGGER_NAME = "HTTP-Trigger"  # Your trigger name (from your Logic App definition)

### Helper Class - Logic Apps Integration

In [ ]:
# Logic App Integration class
class LogicAppsIntegration:
    """
    This class uses Logic Apps callback URL as-is.
    """
    
    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.credential = credential
        self.logic_client = LogicManagementClient(credential, subscription_id)
    
    def get_callback_url(self, logic_app_name: str, trigger_name: str) -> str:
        """Get the actual callback URL from Logic App."""
        try:
            callback = self.logic_client.workflow_triggers.list_callback_url(
                resource_group_name = self.resource_group,
                workflow_name = logic_app_name,
                trigger_name = trigger_name,
            )
            
            if callback.value is None:
                raise ValueError(f"No callback URL returned for Logic App '{logic_app_name}'.")
            
            print(f"Retrieved Logic Apps callback URL")
            return callback.value
            
        except Exception as e:
            print(f"Error getting callback URL: {e}")
            raise
    
    def create_direct_openapi_spec(self, callback_url: str) -> Dict[str, Any]:
        """
        Create a minimal OpenAPI 3.0 spec that uses the callback URL directly.
        """
        print("Creating direct OpenAPI spec using callback URL...")
        
        openapi_spec = {
            "openapi": "3.0.0",
            "info": {
                "title": "Logic App Weather API",
                "version": "1.0.0",
                "description": "Direct access to Logic App via callback URL"
            },
            "servers": [{"url": callback_url.split('?')[0]}], # Callback URL contains auth params
            "paths": {
                "/": {
                    "post": {
                        "operationId": "get_weather",
                        "summary": "Get weather information",
                        "description": "Get weather forecast for a location",
                        "parameters": self._extract_query_params(callback_url),
                        "requestBody": {
                            "required": True,
                            "content": {
                                "application/json": {
                                    "schema": {
                                        "type": "object",
                                        "properties": {
                                            "Location": {
                                                "type": "string",
                                                "description": "The location to get weather for",
                                                "example": "London"
                                            }
                                        },
                                        "required": ["Location"]
                                    }
                                }
                            }
                        },
                        "responses": {
                            "200": {
                                "description": "Weather information",
                                "content": {
                                    "application/json": {
                                        "schema": {
                                            "type": "object",
                                            "description": "Weather forecast data"
                                        }
                                    }
                                }
                            },
                            "default": {
                                "description": "Error response",
                                "content": {
                                    "application/json": {
                                        "schema": {
                                            "type": "object"
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        
        print("Successfully created OpenAPI spec.")
        return openapi_spec
    
    def _extract_query_params(self, callback_url: str) -> list:
        """Extract query parameters from callback URL and make them fixed parameters."""
        from urllib.parse import urlparse, parse_qs
        
        parsed = urlparse(callback_url)
        query_params = parse_qs(parsed.query)
        
        parameters = []
        for param_name, param_values in query_params.items():
            if param_values:
                parameters.append({
                    "name": param_name,
                    "in": "query",
                    "required": True,
                    "schema": {
                        "type": "string",
                        "default": param_values[0]
                    }
                })
        
        print(f"Extracted {len(parameters)} query parameters from callback URL")
        return parameters
    
    def create_openapi_tool(self, logic_app_name: str, trigger_name: str,
                          tool_name: str = None,
                          tool_description: str = None) -> tuple[OpenApiTool, str]:
        """
        Create OpenAPI tool using direct callback URL approach.
        """
        # Get the actual callback URL
        callback_url = self.get_callback_url(logic_app_name, trigger_name)
        
        # Create a minimal OpenAPI spec that uses this URL directly
        openapi_spec = self.create_direct_openapi_spec(callback_url)
        
        # Set defaults
        if not tool_name:
            tool_name = "get_weather"
        if not tool_description:
            tool_description = "Get weather forecast for any location"
        
        # Use anonymous auth as callback URL contains auth
        auth = OpenApiAnonymousAuthDetails()
        
        # Create the tool
        tool = OpenApiTool(
            name = tool_name,
            spec = openapi_spec,
            description = tool_description,
            auth = auth
        )
        
        print(f"Created OpenAPI tool: {tool_name}")
        return tool, tool_name

print("Logic App Integration class was successfully defined")

Logic App Integration class was successfully defined


### Logic Apps Tool Setup

In [4]:
# Initialise Logic App integration
print("Initialising Logic App integration...")
logic_integration = LogicAppsIntegration(
    subscription_id = SUBSCRIPTION_ID,
    resource_group = RESOURCE_GROUP
)

print("Logic App integration initialised successfully")

Initialising Logic App integration...
Logic App integration initialised successfully


In [5]:
# Create OpenAPI Tool
print("Creating OpenAPI tool with direct callback approach...")
openapi_tool, tool_name = logic_integration.create_openapi_tool(
    logic_app_name = LOGIC_APP_NAME,
    trigger_name = TRIGGER_NAME,
    tool_name = "get_weather",
    tool_description = "Get weather forecast for any location"
)

print(f"OpenAPI tool created successfully: {tool_name}")

Creating OpenAPI tool with direct callback approach...
Retrieved Logic Apps callback URL
Creating direct OpenAPI spec using callback URL...
Extracted 4 query parameters from callback URL
Successfully created OpenAPI spec.
Created OpenAPI tool: get_weather
OpenAPI tool created successfully: get_weather


### AI Agent Setup

In [6]:
# Initialise AI Project Client
print("Initialising AI Project client...")
project_client = AIProjectClient(
    endpoint = PROJECT_ENDPOINT,
    credential = DefaultAzureCredential(),
)

agents_client = project_client.agents
print("AI Project client initialised successfully")

Initialising AI Project client...
AI Project client initialised successfully


In [7]:
# Create AI Agent
print("Creating weather agent...")
agent = agents_client.create_agent(
    model = MODEL_DEPLOYMENT,
    name = "weather-agent",
    instructions = "You are a helpful weather assistant. When asked about weather, use the get_weather tool with the location provided by the user.",
    tools = openapi_tool.definitions,
)

print(f"Created agent successfully, ID: {agent.id}")

Creating weather agent...
Created agent successfully, ID: asst_IlYsLmuOplq5b8sZqIGqUltn


In [8]:
# Create Thread
print("Creating conversation thread...")
thread = agents_client.threads.create()

print(f"Created thread successfully, ID: {thread.id}")

Creating conversation thread...
Created thread successfully, ID: thread_6HPOvvzOjJjQb8jwWIHoR6Q5


In [9]:
# Create Message and Run
print("Creating message and running agent...")
message = agents_client.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "What's the weather in London?",
)

print(f"Created message, ID: {message.id}")

# Create and process run
run = agents_client.runs.create_and_process(
    thread_id = thread.id,
    agent_id = agent.id
)

print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Creating message and running agent...
Created message, ID: msg_77BsMUYeSmGc4JUvSGsa7nzf
Run finished with status: RunStatus.COMPLETED


In [10]:
# Display Results
print("Displaying conversation results...")
messages = agents_client.messages.list(thread_id=thread.id)
messages_list = list(messages)
print(f"Found {len(messages_list)} messages")
print("-" * 50)

for i, msg in enumerate(reversed(messages_list)):  # Reverse to show chronological order
    role = msg.role.capitalize()
    
    if hasattr(msg, 'content') and msg.content:
        try:
            if hasattr(msg.content[0], 'text') and hasattr(msg.content[0].text, 'value'):
                content = msg.content[0].text.value
            else:
                content = str(msg.content[0])
        except (IndexError, AttributeError):
            content = "No readable content"
    else:
        content = "No content"
    
    print(f"{role}: {content}")
    print("-" * 50)

Displaying conversation results...
Found 2 messages
--------------------------------------------------
User: What's the weather in London?
--------------------------------------------------
Assistant: The weather in London is currently cloudy. During the day, the high temperature will be 19°C with wind speeds around 22 km/h coming from the southwest (217°). At night, it will remain cloudy with a low temperature of 17°C and winds around 20 km/h from the south-southwest (202°). The skies are overcast, and there is a chance of light rain. Ultraviolet (UV) levels are low.
--------------------------------------------------


### Housekeeping

In [11]:
# Delete the thread and agent when finished
try:
    # Delete the thread
    agents_client.threads.delete(thread.id)
    print("Deleted thread successfully")

    # Use the existing agents_client (no need for 'with' block)
    agents_client.delete_agent(agent.id)
    print("Deleted weather agent successfully")
    
    # Close the project client
    project_client.close()
    print("Closed project client connection")
    
except Exception as e:
    print(f"Error during cleanup: {e}")

print("Cleanup complete!")

Deleted thread successfully
Deleted weather agent successfully
Closed project client connection
Cleanup complete!
